#### Part 1: Create a framework for the game mechanics/rules ####

In [ ]:
import numpy as np

def othello_game(board_size=4):
    # Validate board size
    if not (isinstance(board_size, int) and board_size > 0 and board_size % 2 == 0):
        raise ValueError("board_size must be an even positive integer.")
    print(f"board_size is valid: {board_size}")

    # Initialize the board
    board = np.zeros((board_size, board_size), dtype=str)
    board[board_size/2-1][board_size/2-1] = board[board_size/2][board_size/2] = 'B'  # Player 1
    board[board_size/2-1][board_size/2] = board[board_size/2][board_size/2-1] = 'W'  # Player 2

    def print_board():
        print("  " + " ".join(str(i) for i in range(board_size)))
        for i in range(board_size):
            print(i, " ".join('B' if cell == 'B' else 'W' if cell == 'W' else '.' for cell in board[i]))

    def is_valid_move(x, y, player):
        if board[x][y] != 0:
            return False
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            found_opponent = False
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    found_opponent = True
                elif board[nx][ny] == player:
                    if found_opponent:
                        return True
                    else:
                        break
                else:
                    break
                nx += dx
                ny += dy
        return False

    def make_move(x, y, player):
        board[x][y] = player
        opponent = -player
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            to_flip = []
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    to_flip.append((nx, ny))
                elif board[nx][ny] == player:
                    for fx, fy in to_flip:
                        board[fx][fy] = player
                        break
                    break
                else:
                    break
                nx += dx
                ny += dy
    current_player = 'B'
    while True:
        print_board()
        print(f"Current player: {current_player}")
        move = input("Enter your move as 'row col' or 'q' to quit: ")
        if move.lower() == 'q':
            print("Game ended.")
            break
        try:
            x, y = map(int, move.split())
            if is_valid_move(x, y, current_player):
                make_move(x, y, current_player)
                current_player = 'W' if current_player == 'B' else 'B'
            else:
                print("Invalid move. Try again.")
        except (ValueError, IndexError):
            print("Invalid input. Please enter row and column numbers.")

othello_game()

  0 1 2 3
0 . . . .
1 . B W .
2 . W B .
3 . . . .
Current player: B
Invalid move. Try again.
  0 1 2 3
0 . . . .
1 . B W .
2 . W B .
3 . . . .
Current player: B
Game ended.


In [ ]:
move = "2 3"
x, y = map(int, move.split())
print(x)
print(y)
